In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.8 MB/s eta 0:00:00


In [2]:
import os
from ultralytics import YOLO
import cv2 as cv
from PIL import Image
import shutil
import random

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
source = "/kaggle/input/traffic-signs-dataset-in-yolo-format/ts/ts"
# shutil.rmtree("/kaggle/working/dataset")
os.makedirs("/kaggle/working/dataset/images/train", exist_ok = True)
os.makedirs("/kaggle/working/dataset/images/val", exist_ok = True)
os.makedirs("/kaggle/working/dataset/labels/train", exist_ok = True)
os.makedirs("/kaggle/working/dataset/labels/val", exist_ok = True)

In [4]:
imgs_list = [f for f in os.listdir(source) if f.endswith(".jpg")]

In [5]:
len(imgs_list) * 2 == len(os.listdir(source))

True

In [6]:
imgs_list = [f for f in os.listdir(source) if f.endswith(".jpg")]
random.shuffle(imgs_list)

split_index = int(len(imgs_list) * 0.8)
train_images = imgs_list[:split_index]
val_images = imgs_list[split_index:]

def move(images, split):
    for img in images:
        shutil.copy(f"{source}/{img}",
                   f"/kaggle/working/dataset/images/{split}/{img}")
        
        label = img.replace("jpg", "txt")
        shutil.copy(f"{source}/{img.replace("jpg", "txt")}",
                   f"/kaggle/working/dataset/labels/{split}/{label}")

move(train_images, "train")
move(val_images, "val")

In [7]:
print(len(os.listdir("/kaggle/working/dataset/images/train")))
print(len(os.listdir("/kaggle/working/dataset/labels/train")))
print()
print(len(os.listdir("/kaggle/working/dataset/images/val")))
print(len(os.listdir("/kaggle/working/dataset/labels/val")))

592
592

149
149


In [8]:
%%writefile data.yaml
path: /kaggle/working/dataset
train: images/train
val: images/val

names:
  0: prohibitory
  1: danger
  2: mandatory
  3: other

Writing data.yaml


In [9]:
model = YOLO("yolo26n.pt")
print("Traning the model ...")
model.train(data = "/kaggle/working/data.yaml",
           epochs = 75,
           imgsz = 640)
print("-"*20)

Traning the model ...
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=75, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=Tr

In [10]:
model = YOLO("runs/detect/train/weights/best.pt")
results = model.predict(
    source="/kaggle/input/traffic-signs-dataset-in-yolo-format/traffic-sign-to-test.mp4",  
    save=True,                  
    save_txt=False,             
    device=0    
)


WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/56) /kaggle/input/traffic-signs-dataset-in-yolo-format/traffic-sign-to-test.mp4: 384x640 2 prohibitorys, 72.7ms
video 1/1 (frame 2/56) /kaggle/input/traffic-signs-dataset-in-yolo-format/traffic-sign-to-test.mp4: 384x640 3 prohibitorys, 12.0ms
video 1/1 (frame 3/56) /kaggle/input/traffic-signs-dataset-in-yolo-format/traffic-sign-to-test.mp4: 384x640 2 prohibitorys, 14.5ms
video 1/1 (frame 4/56) /kaggle/input/traffic-signs-dataset-in-yol